In [1]:
import numpy as np
import pickle
import cv2
import os
import matplotlib.pyplot as plt
import math

In [9]:
session = 'video_Jonathan_sc92'
video1 = 'left_edited'
video2 = 'right_edited'
inputs_path = '/Users/sidoodler/projects/asl/datasets/video_asl/inputs'
results_path = '/Users/sidoodler/projects/asl/datasets/video_asl/results'
codes_path = '/Users/sidoodler/projects/asl/codes'

# FOR CAROLINE_SESSION_1: -3, FOR CAROLINE_SESSION_2: -3, FOR CAROLINE_SESSION_3: 0, FOR CAROLINE_SESSION_4: 0
# FOR CAROLINE_SESSION_5: -1, FOR CAROLINE_SESSION_6: -1, FOR HANNA_SESSION_1: -1, FOR HANNA_SESSION_2: 1
# FOR HANNA_SESSION_3: 0, FOR HANNA_SESSION_4: 2, FOR HANNA_SESSION_7: 2
# FOR THE PUBLIC DATASET: 0

frames_sync_diff = 0

frame_index_pairs_file = f'{results_path}/{session}/frame_index_pairs.txt'
frame_index_pairs = open(frame_index_pairs_file, "r")
frame_index_pairs = np.array([int(i) for i in frame_index_pairs.read().split()])
frame_index_pairs = np.reshape(frame_index_pairs, (int(len(frame_index_pairs)/2), 2))

dim_and_crop_file1 = f'{results_path}/{session}/{video1}/dim_and_crop.txt'
dim_and_crop = open(dim_and_crop_file1, "r")
dim_and_crop = np.array([int(i) for i in dim_and_crop.read().split()])

u1, d1, l1, r1 = dim_and_crop[2], dim_and_crop[3], dim_and_crop[4], dim_and_crop[5]

dim_and_crop_file2 = f'{results_path}/{session}/{video2}/dim_and_crop.txt'
dim_and_crop = open(dim_and_crop_file2, "r")
dim_and_crop = np.array([int(i) for i in dim_and_crop.read().split()])

u2, d2, l2, r2 = dim_and_crop[2], dim_and_crop[3], dim_and_crop[4], dim_and_crop[5]

In [10]:
# CONCATENATION OF IMAGE_LEFT, IMAGE_RIGHT AND OUTPUT IMAGE

def return_images_count(video):
    
    d = f"{results_path}/{session}/{video}/images/"
    images_count = 0
    
    for path in os.listdir(d):
        if os.path.isfile(os.path.join(d, path)):
            images_count += 1
            
    return images_count


if os.path.isdir(f'{results_path}/{session}/composite_images') == False:
        os.mkdir(f'{results_path}/{session}/composite_images')

images_left_count = return_images_count(video1)
images_right_count = return_images_count(video2)

i = 0
while True:
    
    # img1 is the left mediapipe annotated output, img2 is the right mediapipe annotated output and img3 is the step 4 final output
    
    if frames_sync_diff >= 0:
        
        a = frame_index_pairs[i,1] > images_right_count
        b = frame_index_pairs[i,1] > images_left_count - frames_sync_diff
        
        if a or b:
            break
            
        img1 = cv2.imread(f'{results_path}/{session}/{video1}/images/{frame_index_pairs[i,1] + frames_sync_diff}.png')
        img2 = cv2.imread(f'{results_path}/{session}/{video2}/images/{frame_index_pairs[i,1]}.png')
        
        img1 = img1[:,l1:r1,:]
        img2 = img2[:,l2:r2,:]
            
    else:
        
        a = frame_index_pairs[i,1] > images_right_count + frames_sync_diff
        b = frame_index_pairs[i,1] > images_left_count
        
        if a or b:
            break
            
        img1 = cv2.imread(f'{results_path}/{session}/{video1}/images/{frame_index_pairs[i,1]}.png')
        img2 = cv2.imread(f'{results_path}/{session}/{video2}/images/{frame_index_pairs[i,1] - frames_sync_diff}.png')
        
        img1 = img1[:,l1:r1,:]
        img2 = img2[:,l2:r2,:]
        
    img3 = cv2.imread(f'{results_path}/{session}/final_keypoint_figures_300/{i+1}_front.png')
        
    channel0 = cv2.resize(img3[:,:,0], (480, 480))
    channel1 = cv2.resize(img3[:,:,1], (480, 480))
    channel2 = cv2.resize(img3[:,:,2], (480, 480))

    channel0 = np.reshape(channel0, (480, 480, 1))
    channel1 = np.reshape(channel1, (480, 480, 1))
    channel2 = np.reshape(channel2, (480, 480, 1))

    img3 = np.concatenate((channel0, channel1, channel2), axis=2)

    img = np.concatenate((img1, img2, img3), axis=1)

    cv2.imwrite(f'{results_path}/{session}/composite_images/{i+1}.png', img)
    
    i += 1